In [1]:
from setup import D

synth_count = 10
d = D(
    synth_count=synth_count,
    context='opl',
)


In [2]:
from enum import Enum
import time


class Event():
    def __init__(self, note, scale, timing, dur, synth_idx):
        self.note = note
        self.scale = scale
        self.timing = timing
        self.dur = dur
        self.synth_idx = synth_idx

class Notes(str, Enum):
    REST = '.'

class Events():
    def __init__(self, stream, scale):
        # NOTE:
        # - scale is global
        # - notation for duration is not finalized
        # - sustain is same as duration
        # - bpm is global
        # - no info about synth indices
        self.events = []
        cur_beat = 0
        for notes in stream:
            # convert to list
            chord = []
            dur = 1
            if isinstance(notes, str) and (notes != Notes.REST):
                notes, dur = notes.split(':')
                dur = int(dur)
                notes = eval(notes)
            match notes:
                case int():
                    chord = [notes]
                case Notes.REST:
                    chord = [Notes.REST]
                case tuple():
                    chord = list(notes)
                case _:
                    print(f'Wrong type for notes {notes}: {type(notes)}')
                    assert False

            for note_idx, note in enumerate(chord):
                self.events.append(Event(note, scale, cur_beat, dur,
                                         synth_idx=note_idx))
            cur_beat += dur


def play(events: Events, client,
         dur=0.25,
         sus=None,
         voice_amps=[],
         # FIXME:
         # delay=None,
         # synth_idx=[0],
         # rep=1,
         ):

    # NOTE: crutch; play random note silently, so that the timbre doesn't jump on first note
    # FIXME: depends on synth
    # timetag = time.time()
    # for note_idx in range(synth_count):
    #     client.send_note(
    #             note_idx, freq=C4_FREQ,
    #             dur=0.1, amp=0.0,
    #             timetag=timetag, channel=0)
    # time.sleep(0.1)

    # FIXME:
    # if isinstance(rep, list):
    #     assert len(synth_idx) == len(rep)
    # else:
    #     rep = [rep] * len(synth_idx)
    # last_reps = [0] * len(synth_idx)

    cur_beat = 0
    for event in events.events:
        assert event.synth_idx < synth_count

        # NOTE: assuming events are sorted by timing
        assert cur_beat <= event.timing
        if event.timing > cur_beat:
            sleep_dur = dur * (event.timing - cur_beat)
            time.sleep(sleep_dur)
            cur_beat = event.timing

        send_note_dur = dur
        if sus:
            send_note_dur = sus
        send_note_dur *= event.dur

        amp = 1.0
        if voice_amps:
            amp *= voice_amps[event.synth_idx]  # TODO: should be voice_idx

        freq = event.scale.get_freq(event.note)
        timetag = time.time()
        client.send_note(
            # FIXME: synth_idx[note_idx] + last_reps[note_idx],
            event.synth_idx,
            freq=freq,
            dur=send_note_dur,
            amp=amp,
            timetag=timetag,
            channel=0,  # TODO: for MPE with channels use channel=note_idx
            )
        # FIXME:
        # last_reps[note_idx] = (last_reps[note_idx] + 1) % rep[note_idx]
        # if delay:
            # time.sleep(delay)
    time.sleep(dur)


In [ ]:
# TODO
# tetracot MOS scales
# tetracot[7], tetracot[13], tetracot[20]
generator = Cents(176)  # cents